## AttacKG result eval
- 主詞: 'type': 'executable'
- 受詞: rc.local /proc/self/exe https:// 61.147.91.53

In [32]:
import json
import networkx as nx
dot_path = "./results_AttacKG/Dofloo-SyscallParty_output.pdf.dot"
json_path = "./results_AttacKG/Dofloo-SyscallParty_output.pdf_artifacts.json"

def read_json(json_path: str) -> dict:
    ''' merge node data into a list '''
    with open(json_path, 'r', encoding='utf-8') as f:
        jsdata:dict = json.load(f)
    nodes_info: dict[str,list] = {}
    for nodeid, val in jsdata.items():
        nodes_info[nodeid] = {
            'type': val['type'],
            'data': sorted(val['nlp'] + val['ioc'])
        }
    del jsdata
    return nodes_info

def read_dot(dot_path: str) -> nx.Graph:
    G = nx.Graph(nx.nx_pydot.read_dot(dot_path))
    return G

def search(input_list: list[str]):
    '''return the first hit item''' #todo return all hit items
    target = ['rc.local', '/proc/self/exe', '61.147.91.53']
    for i in input_list:
        for t in target:
            if i.find(t) != -1:
                return t
            
def get_scrnode_from_dstnode(G: nx.Graph, nid: str) -> list:
    scr_lst = []
    for e in G.edges():
        if e[1] == nid:
            scr_lst.append(e[0])
    return scr_lst

In [34]:
G = read_dot(dot_path)
# for e in G.edges():
#     print(e)

# todo: 計算node edge個數
print(f"num of object-like items {len(G.nodes())}, num of dependencies {len(G.edges())}")

nodes_info = read_json(json_path)
hit_objs = set()
print("\nContains actions:")
for nid, n in nodes_info.items():
    # print(n)
    res = search(n['data'])
    if res:
        hit_objs.add(res)
        # print(res)
        scr_nid_lst = get_scrnode_from_dstnode(G, nid)
        for src_nid in scr_nid_lst:
            scrnode = nodes_info[src_nid]
            print(f"{scrnode} -> {res}")
print(f"\nContains {len(hit_objs)} matched objects in report: \n{hit_objs}")

num of object-like items 191, num of dependencies 102

Contains actions:
{'type': 'file', 'data': ['file']} -> rc.local
{'type': 'executable', 'data': ['command', 'malware']} -> rc.local
{'type': 'executable', 'data': ['commands']} -> rc.local

Contains 3 matched objects in report: 
{'61.147.91.53', 'rc.local', '/proc/self/exe'}


In [29]:
G.edges()

EdgeView([('14', '12'), ('12', '34'), ('34', '31'), ('31', '36'), ('44', '51'), ('51', '78'), ('78', '54'), ('54', '56'), ('176', '192'), ('192', '166'), ('166', '195'), ('195', '187'), ('187', '204'), ('204', '171'), ('171', '210'), ('230', '245'), ('245', '241'), ('280', '276'), ('299', '297'), ('307', '327'), ('327', '330'), ('330', '325'), ('338', '335'), ('335', '359'), ('359', '367'), ('425', '444'), ('548', '533'), ('550', '554'), ('559', '567'), ('592', '603'), ('603', '615'), ('619', '651'), ('677', '692'), ('692', '681'), ('730', '743'), ('822', '824'), ('870', '899'), ('899', '897'), ('897', '875'), ('875', '884'), ('913', '917'), ('917', '912'), ('912', '924'), ('939', '948'), ('948', '952'), ('952', '965'), ('1007', '986'), ('1017', '1023'), ('1064', '1072'), ('1105', '1100'), ('1100', '1115'), ('1115', '1111'), ('1124', '1123'), ('1164', '1152'), ('1152', '1181'), ('1181', '1173'), ('1173', '1153'), ('1153', '1161'), ('1255', '1261'), ('1293', '1278'), ('1278', '1280'), (

In [18]:
def get_scrnode_from_dstnode(G: nx.Graph, nid: str) -> list:
    scr_lst = []
    for e in G.edges():
        if e[1] == nid:
            scr_lst.append(e[0])
    return scr_lst

get_scrnode_from_dstnode(G, '1638')

['1617']